In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Import train set

In [ ]:
df = pd.read_csv('../input/train.csv', nrows=10_000_000)
df.head()

Feature exploration

In [ ]:
df.time_to_failure.nunique()  # oh...

In [ ]:
df.acoustic_data.unique()

How do submissions look?

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')
sub.head()

In [ ]:
sub.shape

What about the test sets?

In [ ]:
test1 = pd.read_csv('../input/test/seg_00030f.csv')
test1.shape

In [ ]:
test1.acoustic_data.nunique(), test1.acoustic_data.unique()

In [ ]:
test1.head()

So, what are we predicting?

In [ ]:
import os
test_files = list(os.listdir("../input/test"))
len(test_files), test_files[:5]

Need some changes

In [ ]:
test1.acoustic_data.unique().mean()

Make a new Dataframe from all the test data

In [ ]:
%%time

folder = '../input/test/'
medians = []
test_files_feat = []

for file_path in test_files:
    test_files_feat.append(file_path[:-4])
    path = folder + file_path
    test_df = pd.read_csv(path)
    medians.append(int(test_df.acoustic_data.unique().mean()))

In [ ]:
len(medians), medians[:5], len(test_files_feat), test_files_feat[:5]

In [ ]:
test = pd.DataFrame({ 'seg_id' : test_files_feat, 'acoustic_data' : medians })
test.head()

Model

In [ ]:
from xgboost import XGBRegressor
#from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as mae

In [ ]:
X, y = df.drop('time_to_failure', axis=1), df.time_to_failure

In [ ]:
%%time
#rf = RandomForestRegressor(n_estimators=20)  #, criterion='mae')
#rf.fit(X, y)
xgb = XGBRegressor(max_depth=5, n_estimators=30, n_jobs=-1)
xgb.fit(X, y)

In [ ]:
#rf.predict(test.drop('seg_id', axis=1))
xgb.predict(test.drop('seg_id', axis=1))

In [ ]:
#test['time_to_failure'] = pd.Series(rf.predict(test.drop('seg_id', axis=1)))
test['time_to_failure'] = pd.Series(xgb.predict(test.drop('seg_id', axis=1)))
test.head()

In [ ]:
subm = pd.merge(
    sub.drop('time_to_failure', axis=1),
    test.drop('acoustic_data', axis=1),
    on='seg_id'
)
subm.head()

In [ ]:
subm.to_csv('submission.csv', index=False)